# Orange brix Analysis

## Import thư viện

## Lưu ý: cài thư viện matplotlib bản 3.7.3 để không bị lỗi plot

In [ ]:
# !pip uninstall matplotlib
# !pip install matplotlib==3.7.3

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from enum import Enum
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
import pickle

import os

In [ ]:
plt.style.use("fivethirtyeight")

## Các cài đặt khác

In [ ]:
# Biến GENERATE_PLOTS dùng để bật tắt việc vẽ đồ thị.
# Tắt đi để tăng tốc chạy test. (không chạy những hàm plot)
GENERATE_PLOTS: bool = False

# Biến boolean để bật tắt việc traing model khi ấn "Run all"
TRAIN_MODEL_1: bool = False
TRAIN_MODEL_2: bool = False
TRAIN_MODEL_3: bool = False
TRAIN_MODEL_4: bool = False
TRAIN_MODEL_5: bool = False

## Tạo các Class

### Class Utilities

In [ ]:
# Class Utilities có các phương thức tiện ích.
# Các class khác muốn sử dụng các tiện ích này thì
# chỉ cần kế thừa class Utilities này.

class Utilities:
    def __init__(self):
        pass

    def get_categorical_numeric_cols(self, dataframe: pd.DataFrame):
        if len(dataframe) == 0:
            raise Exception("No records found!")

        numeric_cols = []
        categorical_cols = []

        for col in dataframe.columns:
            if isinstance(dataframe[col][0], (bool, np.bool_)) or not isinstance(
                dataframe[col][0], (int, float, np.int8, np.int16, np.int32, np.int64, np.float16, np.float32, np.float64)
            ):
                categorical_cols.append(col)
            else:
                numeric_cols.append(col)

        return (categorical_cols, numeric_cols)

### Enum ScalerType

In [ ]:
# Khai báo Enum của các loại Scaler data từ submoule
# preprocessing của thư viện sklearn.

from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    RobustScaler,
    StandardScaler,
)


class ScalerType(Enum):
    Raw_ = None
    MaxAbsScaler_ = MaxAbsScaler
    MinMaxScaler_ = MinMaxScaler
    Normalizer_ = Normalizer
    RobustScaler_ = RobustScaler
    StandardScaler_ = StandardScaler

### Class Dataset

In [ ]:
# Class Dataset chứa 1 dataframe và các metadata của
# dataframe này (tên, loại scaler, tên cột biến định
# danh, tên cột biến định lượng)


class Dataset(Utilities):
    def __init__(
        self,
        name: str,
        dataframe: pd.DataFrame,
        scaler_type: ScalerType = ScalerType.Raw_,
    ):
        self.name: str = name
        self.dataframe: pd.DataFrame = dataframe
        self.categorical_cols: list[str] = []
        self.numeric_cols: list[str] = []
        self.scaler_type: ScalerType = scaler_type

        self.categorical_cols, self.numeric_cols = self.get_categorical_numeric_cols(
            self.dataframe
        )

    def get_name(self) -> str:
        return self.name

    def get_dataframe(self) -> pd.DataFrame:
        return self.dataframe

    def detect_categorical_numeric_cols(self):
        if len(self.dataframe) == 0:
            raise Exception("No records found!")

        self.numeric_cols = []
        self.categorical_cols = []

        for col in self.dataframe.columns:
            try:
                float(self.dataframe[col][0])
                self.numeric_cols.append(col)
            except:
                self.categorical_cols.append(col)

    def get_numeric_cols(self) -> list[str]:
        return self.numeric_cols

    def get_numeric_dataframe(self) -> pd.DataFrame:
        return self.dataframe[self.numeric_cols]

    def get_categorical_dataframe(self) -> pd.DataFrame:
        return self.dataframe[self.categorical_cols]

    def get_scaler_type(self) -> ScalerType:
        return self.scaler_type

    def concat_dataframe(self, additional_dataframe: pd.DataFrame):
        if len(additional_dataframe) == len(self.dataframe):
            self.dataframe = pd.concat([self.dataframe, additional_dataframe], axis=1)
            self.detect_categorical_numeric_cols()

    def get_one_hot_vectorized_dataset(
        self, categorical_col: str, print_name: bool = True
    ):
        if categorical_col in self.categorical_cols:
            dummies_dataframe = pd.get_dummies(self.dataframe[categorical_col])
            dummies_dataframe = pd.concat([dummies_dataframe, self.dataframe], axis=1)
            new_dataset_name = self.name + "_ohv_" + categorical_col
            if print_name:
                print(new_dataset_name)
            return Dataset(
                name=new_dataset_name,
                dataframe=dummies_dataframe,
                scaler_type=self.scaler_type,
            )
        elif categorical_col in self.numeric_cols:
            print("Records for input column name are not categorical!")
        else:
            print("Input column name does not exists!")

### Enum ModelType

In [ ]:
# ModelType là Enum khai báo các loại model từ thư viện
# sklearn và một số mô hình từ nguồn khác (xgb, lightgbm).
# Mỗi loại model được khai báo một bộ tham số đi kèm.


class ModelType(Enum):
    LINEAR_REGRESSION: dict = {}
    SVR: dict = {
        "kernel": "rbf",  # | linear, poly, rbf
        "degree": 3,
        "gamma": "scale",
        "coef0": 0,
        "tol": 1e-3,
        "C": 1.0,  # 0.1 - 2 step 0.1
        "epsilon": 0.1,
    }
    DECISION_TREE: dict = {
        "criterion": "squared_error",
        "splitter": "best",
        "max_depth": None,  # 1 - number of feature
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "min_weight_fraction_leaf": 0.0,
        "max_features": None,  # int 2->5 {“sqrt”, “log2”}, default=None
        "random_state": 42,
        "max_leaf_nodes": None,
        "min_impurity_decrease": 0,
    }
    RANDOM_FOREST: dict = {
        "n_estimators": 100,  # int 100-1000 (step 100)
        "criterion": "squared_error",
        "max_depth": None,  # 1 - number of feature
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "min_weight_fraction_leaf": 0.0,
        "max_features": 1.0,  # int 2->5 {“sqrt”, “log2”}, default=None
        "max_leaf_nodes": None,
        "min_impurity_decrease": 0.0,
    }
    LIGHT_GBM: dict = {
        "boosting_type": "gbdt",
        "num_leaves": 31,
        "max_depth": -1,
        "learning_rate": 0.1,
        "n_estimator": 100,  # int 100-1000 (step 100)
        "verbosity": -1,
    }
    XGB: dict = {
        "objective": "reg:linear",
        "n_estimators": 10,  # int 100-1000 (step 100)
        "seed": 123,
        "verbosity": 0,
    }

### Class Result

In [ ]:
# Class Result dùng để chứa kết quả huấn luyện và các
# thông tin liên quan của một bộ (mô hình + dữ liệu).

# Một kết quả gồm:
# - Tên bộ dữ liệu
# - Tên của bộ tham số pretrained
# - Loại mô hình
# - Loại scaler
# - Kết quả r^2
# - Kết quả mse
# - Các tham số pretrained


class Result:
    def __init__(
        self,
        dataset_name: str,
        pretrained_model_features: str,
        model_type: ModelType,
        scaler_type: str,
        r2: float,
        mse: float,
        params: str,
    ):
        self.result = [
            dataset_name,
            pretrained_model_features,
            model_type,
            scaler_type,
            r2,
            mse,
            params,
        ]

    def get_result(self):
        return self.result

### Class ResultFrame

In [ ]:
# Class ResultFrame chứa bảng chứa tất cả kết quả của
# một vòng huấn luyện mô hình.
# ResultFrame chứa một pd.DataFrame nên có thể hỗ trợ
# việc sắp xếp, lọc, xuất kết quả...


class ResultFrame:
    def __init__(self):
        self.start_up()

    def start_up(self):
        self.result_frame: pd.DataFrame = pd.DataFrame()
        self.count = 0
        self.result_frame["dataset_name"] = []
        self.result_frame["pretrained_model_features"] = []
        self.result_frame["model_type"] = []
        self.result_frame["scaler_type"] = []
        self.result_frame["r2"] = []
        self.result_frame["mse"] = []
        self.result_frame["params"] = []

    def add_result(self, new_result: Result):
        self.result_frame.loc[self.count] = new_result.result
        self.count += 1

    def reset_result(self):
        self.start_up()

    def display_result(self, display_rows: int = 30):
        self.result_frame.sort_values(["r2", "mse"], ascending=False, inplace=True)
        if display_rows < 0:
            display_rows = 30
        display(self.result_frame.head(display_rows))

    def save_result(self, filename=None, overwrite=False):
        if self.result_frame.shape[0] == 0:
            print("Result has no records!")
        elif not filename:
            filename = (
                self.result_frame["dataset_name"][0]
                + "_"
                + self.result_frame["pretrained_model_features"][0]
                + ".xlsx"
            )
            self.result_frame.to_excel(filename)
        else:
            file_exists = os.path.isfile(os.path.join(os.getcwd(), filename))
            if file_exists and not overwrite:
                print("Error: CANNOT SAVE FILE.")
                print("A file with the same name already exists.")
                print(
                    "Set overwrite to True to overwrite existed file or change file name!"
                )
            else:
                self.result_frame.to_excel(filename)

### Class Model (là class quan trọng nhất trong notebook này)

In [ ]:
# Class Model dùng để tiền xử lý dữ liệu, huấn luyện và
# đánh giá mô hình với bộ dữ liệu đã cho ở đầu vào.

# Tham số đầu vào:
# - dataset_name: tên của Dataset
# - pretrained_model_features: tên của bộ tham số pretrained
# - model_type: loại Model
# - x_cols: một list chứa tên các cột sử dụng làm biến giải thích
# - y_cols: một list chứa tên các cột biến phản hồi
# - dataset: dataset mình muốn xử lý (dataset này có thể có
#       các cột không liên quan, các dữ liệu cho X và Y sẽ được
#       trích xuất tự động và các dữ liệu không liên quan sẽ bị loại bỏ)
# - scaler_type: loại scaler cho dữ liệu

# Cách sử dụng
# 1. Khởi tạo bảng kết quả: my_result_frame = ResultFrame()
# 2. Khai báo cột x dưới dạng list (các biến giải thích)
# 3. Khai báo cột y dưới dạng list (các biến phản hồi)
# 4. Tạo instance Model, khai báo các biến đầu vào (giả sử đặt là my_model)
# 5. Huấn luyện và add kết quả vào bảng kết quả (sử dụng 1 dòng duy nhất
#     my_result_frame.add_result(my_model.evaluate())
# 6. Hiển thị kết quả
#     my_result_frame.display_result()
# 7. Lưu kết quả vào file "my_result.xlsx"
#     my_result_frame.save_result("my_result.xlsx")

class Model(Utilities):
    def __init__(
        self,
        dataset_name: str,
        pretrained_model_features: list[str],
        model_type: ModelType,
        x_cols: list[str],
        y_cols: list[str],
        dataset: Dataset,
        scaler_type: ScalerType = ScalerType.Raw_,
    ):

        if len(x_cols) == 0 or len(y_cols) == 0:
            raise Exception("Data columns are not specified!")

        self.dataset_name: str = dataset_name
        self.model_type: ModelType = model_type
        self.pretrained_model_features = str(pretrained_model_features)
        self.scaler_type: ScalerType = scaler_type
        self.scaler = self.get_scaler(self.scaler_type)  # could be None
        self.x_dataset: Dataset = None
        self.y_dataset: Dataset = None
        self.inner_model = None
        self.r2 = None
        self.mse = None

        if self.model_type != ModelType.LINEAR_REGRESSION:
            self.params = self.model_type.value
        else:
            self.params = {}

        self.preprocess_data(x_cols, y_cols, dataset)

    # Extract data from x_cols and y_cols
    # Redetect categorical and numeric columns
    def preprocess_data(self, x_cols: list[str], y_cols: list[str], dataset: Dataset):
        x_dataset_new_name = dataset.get_name() + "_x"
        y_dataset_new_name = dataset.get_name() + "_y"
        scaler_type = dataset.get_scaler_type()
        self.x_dataset = Dataset(
            name=x_dataset_new_name,
            dataframe=dataset.get_dataframe()[x_cols],
            scaler_type=scaler_type,
        )
        self.y_dataset = Dataset(
            name=y_dataset_new_name,
            dataframe=dataset.get_dataframe()[y_cols],
            scaler_type=scaler_type,
        )

        self.process_x_dataset()

    # arrange [numeric_cols, categorical_cols]
    def process_x_dataset(self):
        new_name = self.x_dataset.get_name()
        numeric_dataframe = self.x_dataset.get_numeric_dataframe()
        categorical_dataframe = self.x_dataset.get_categorical_dataframe()
        scaler_type = self.x_dataset.get_scaler_type()
        self.x_dataset = Dataset(
            name=new_name,
            dataframe=pd.concat([numeric_dataframe, categorical_dataframe], axis=1),
            scaler_type=scaler_type,
        )

    def get_x_list_index_numeric_cols(self):
        return list(range(len(self.x_dataset.get_numeric_cols())))

    def fit_scaler_and_scale_dataframe(self, X: pd.DataFrame) -> pd.DataFrame:
        if self.scaler is not None:
            list_index = self.get_x_list_index_numeric_cols()
            X_numeric = X[:, list_index]
            X_categorical = X[:, len(list_index) :].astype(int)
            self.scaler.fit(X_numeric)
            transformed_X = self.scaler.transform(X_numeric)
            transformed_X = np.concatenate([transformed_X, X_categorical], axis=1)
        else:
            transformed_X = X

        return transformed_X

    def get_x_dataframe(self):
        return self.x_dataset.get_dataframe()

    def get_y_dataframe(self):
        return self.y_dataset.get_dataframe()

    def get_scaler(self, scaler_type: ScalerType):
        if scaler_type != ScalerType.Raw_:
            return scaler_type.value()

    def display_progress(self):
        print(
            f"Training | ModelType: {self.model_type} | Datasetname: {self.dataset_name} | ScalerType: {self.scaler_type} | Params: {self.params}"
        )
        print(f"R2: {self.r2} | MSE: {self.mse}")

    def fit(self, X=None, Y=None):
        if X is None:
            X = np.array(self.x_dataset.get_dataframe())
            Y = np.array(self.y_dataset.get_dataframe())

        model_family = self.model_type.__str__().split(".")[-1]

        Y = np.squeeze(Y)

        # need to scale
        transformed_X = self.fit_scaler_and_scale_dataframe(X)

        if model_family == "SVR":
            self.model = SVR(**(self.params)).fit(transformed_X, Y)
        elif model_family == "LINEAR_REGRESSION":
            self.model = LinearRegression(**(self.params)).fit(transformed_X, Y)
        elif model_family == "DECISION_TREE":
            self.model = DecisionTreeRegressor(**(self.params)).fit(transformed_X, Y)
        elif model_family == "RANDOM_FOREST":
            self.model = RandomForestRegressor(**(self.params)).fit(transformed_X, Y)
        elif model_family == "LIGHT_GBM":
            self.model = LGBMRegressor(**(self.params)).fit(transformed_X, Y)
        elif model_family == "XGB":
            self.model = XGBRegressor(**(self.params)).fit(transformed_X, Y)
        else:
            raise ValueError("Unknown model family!")

    def predict(self, X):
        if self.scaler is not None:
            list_index = self.get_x_list_index_numeric_cols()
            X_numeric = X[:, list_index]
            X_categorical = X[:, len(list_index) :].astype(int)
            transformed_X = self.scaler.transform(X_numeric)
            transformed_X = np.concatenate([transformed_X, X_categorical], axis=1)
        else:
            transformed_X = X
        return self.model.predict(transformed_X)

    def evaluate(self, X=None, Y=None, method="LOOCV"):
        if X is None:
            X = np.array(self.x_dataset.get_dataframe())
            Y = np.array(self.y_dataset.get_dataframe())
        if method == "LOOCV":
            preds = [0] * len(Y)
            total = len(Y)
            kf = KFold(n_splits=total)
            kf.get_n_splits(Y)

        with tqdm(total=total) as pbar:
            for i, (train_index, valid_index) in enumerate(kf.split(X, Y)):
                X_train = X[train_index]
                Y_train = Y[train_index]
                X_valid = X[valid_index]
                Y_valid = Y[valid_index]

                self.fit(X_train, Y_train)

                Y_valid_pred = self.predict(X_valid)
                for j in range(len(valid_index)):
                    index = valid_index[j]
                    value = Y_valid_pred[j]
                    preds[index] = value
                pbar.update(1)

        self.r2 = r2_score(Y, preds)
        self.mse = mean_squared_error(Y, preds)

        self.display_progress()

        # Compile result
        return Result(
            dataset_name=self.dataset_name,
            pretrained_model_features=self.pretrained_model_features,
            model_type=self.model_type.name,
            scaler_type=self.scaler_type.name,
            r2=self.r2,
            mse=self.mse,
            params=self.params,
        )

    def save(self, filename: str):
        pickle.dump(self.inner_model, open(filename, "wb"))

    def load(self, filename: str):
        pickle.dump(self.inner_model, open(filename, "wb"))

    def set_params(self, params: dict):
        self.params = params

### Các biến global trong notebook

In [ ]:
# list_dataset: một dictionary chứa toàn bộ dataset của
# notebook
list_dataset: dict[str, Dataset] = dict()

# result_frame: dùng để lưu các kết quả của việc huấn luyện
# mô hình.
result_frame: ResultFrame = ResultFrame()

# hyper_params: dùng để lưu các tham số cho việc fine-tuning
# mô hình.
hyper_params = {
    "SVR": {"kernel": ["rbf", "linear", "poly"], "C": np.arange(0.1, 2, 0.1)},
    "RANDOM_FOREST": {
        "n_estimators": np.arange(100, 1100, 100),
        "max_depth": [None, 5, 10, 15, 20],
        "max_features": [10, 50, 100, 150, 200, "sqrt", "log2", None],
    },
    "LIGHT_GBM": {
        "max_depth": [-1],  # , 5, 10, 15, 20],
        "learning_rate": [0.01],  # , 0.05, 0.1],
        "min_child_samples": [20],
        "n_estimators": np.arange(800, 1000, 100),
        # "subsample" : [1.0, 0.8, 0.6, 0.5],
        #  "colsample_bytree" : [0.1, 0.2, 0.4, 0.6, 0.8, 1],
        #   "reg_alpha": [0, 0.01, 0.05, 0.1, 0.2],
        # "reg_lambda": [0, 0.01, 0.05, 0.1, 0.2],
    },
}

## Các hàm để vẽ đồ thị, biểu đồ

### plot_numeric_along_dataframe

In [ ]:
# Hàm này vẽ line plot cho tất cả các cột dữ liệu
# định lượng.
# Dữ liệu chạy theo thứ tự lúc import.
# Hàm này để có cái nhìn ban đầu về bộ dữ liệu.

# Hàm này nhận 4 tham số
# - dataset: dataset đầu vào
# - nrow_subplot: số hàng của subplot
# - ncol_subplot: số cột của subplot
# - figsize: để tùy chỉnh kích thước của hình vẽ


def plot_numeric_along_dataframe(
    dataset: Dataset,
    nrow_subplot: int,
    ncol_subplot: int,
    figsize: tuple[int, int],
):
    numeric_df = dataset.get_numeric_dataframe()
    x = np.arange(len(numeric_df))
    figs, axes = plt.subplots(nrow_subplot, ncol_subplot, figsize=figsize)
    for i in range(nrow_subplot):
        for j in range(ncol_subplot):
            ax = axes[i][j]
            index = i * ncol_subplot + j
            ax.plot(x, numeric_df.iloc[:, index])
            ax.set_title(f"{numeric_df.columns[index]} along dataframe", size=15)
            ax.xaxis.set_tick_params(labelsize=12)
            ax.xaxis.set_ticks(np.arange(0, len(numeric_df), 10))
            ax.yaxis.set_tick_params(labelsize=12)

### plot_hist_kde_box_all_columns

In [ ]:
# Hàm này vẽ các 3 plot cho mỗi biến định lượng của dữ liệu
# 3 đồ thị
# - Đồ thị histogram
# - Đồ thị KDE (kerner density estimation)
# - Đồ thị Boxplot

def plot_hist_kde_box_all_columns(dataset: Dataset):

    number_of_numeric_col = len(dataset.numeric_cols)
    figs, axes = plt.subplots(
        number_of_numeric_col, 3, figsize=(30, number_of_numeric_col * 10)
    )

    numeric_dataframe = dataset.get_dataframe()[dataset.numeric_cols]

    for i in range(number_of_numeric_col):
        axes[i][0].set_title(f"{numeric_dataframe.columns[i]} histogram", size=20)
        axes[i][0].yaxis.set_tick_params(labelsize=15)
        axes[i][0].xaxis.set_tick_params(labelsize=15)
        sns.histplot(ax=axes[i][0], data=numeric_dataframe.iloc[:, i])

        axes[i][1].set_title(f"{numeric_dataframe.columns[i]} KDE", size=20)
        axes[i][1].xaxis.set_tick_params(labelsize=15)
        axes[i][1].yaxis.set_tick_params(labelsize=15)
        sns.kdeplot(ax=axes[i][1], data=numeric_dataframe.iloc[:, i])

        axes[i][2].set_title(f"{numeric_dataframe.columns[i]} Boxplot", size=20)
        axes[i][2].xaxis.set_tick_params(labelsize=15)
        axes[i][2].yaxis.set_tick_params(labelsize=15)
        sns.boxplot(ax=axes[i][2], data=numeric_dataframe.iloc[:, i])

### plot_all_columns_group_by_a_categorical_col

In [ ]:
# Hàm plot các cột dữ liệu định lượng được phân chia thành
# các loại từ 01 cột định danh cho trước.

def plot_all_columns_group_by_a_categorical_col(dataset: Dataset, categorical_col: str):

    dataframe = dataset.get_dataframe()
    numeric_dataframe = dataframe[dataset.numeric_cols]
    number_of_numeric_col = len(dataset.numeric_cols)
    figs, axes = plt.subplots(
        number_of_numeric_col, 2, figsize=(20, number_of_numeric_col * 10)
    )
    groups = dataframe[categorical_col].unique()

    for i in range(number_of_numeric_col):
        axes[i][0].set_title(f"{numeric_dataframe.columns[i]} KDE", size=20)
        axes[i][0].xaxis.set_tick_params(labelsize=15)
        axes[i][0].yaxis.set_tick_params(labelsize=15)
        for group in groups:
            sns.kdeplot(
                ax=axes[i][0],
                data=numeric_dataframe[dataframe[categorical_col] == group].iloc[:, i],
                label=f"Type {group}",
            )
        axes[i][0].legend()

        axes[i][1].set_title(f"{numeric_dataframe.columns[i]} Boxplot", size=20)
        axes[i][1].xaxis.set_tick_params(labelsize=15)
        axes[i][1].yaxis.set_tick_params(labelsize=15)
        sns.boxplot(
            ax=axes[i][1],
            hue="type",
            y=numeric_dataframe.columns[i],
            data=dataframe,
        )

### divide_dataframe_with_categorical_col

In [ ]:
# Chia dataframe thành các dataframe theo nhóm của
# một biến định danh cho trước.

def divide_dataframe_with_categorical_col(
    dataset: Dataset, categorical_col: str
) -> dict[str, pd.DataFrame]:
    dataframe = dataset.get_dataframe()
    groups = dataframe[categorical_col].unique()

    result = {}
    for group in groups:
        result[group] = dataframe[dataframe[categorical_col] == group]

    return result

### calculate_p_values_between_partitions

In [ ]:
# Tính p_value của các cột trong bảng, chia theo
# một biến giải thích định danh cho trước.

def calculate_p_values_between_partitions(dataframes, columns_to_compare, threshold=0.05):
    if not 0 < threshold < 1:
        raise Exception("Threshold must be in range (0, 1)")

    print("p_value threshold: ", threshold)
    p_values_greater_threshold = {}
    p_values_smaller_threshold = {}
    for key1, df1 in dataframes.items():
        for key2, df2 in dataframes.items():
            if key1 != key2:
                for col in columns_to_compare:
                    if col in df1 and col in df2:
                        t_stat, p_val = stats.ttest_ind(df1[col], df2[col])
                        if p_val > threshold:
                            p_values_greater_threshold[f"{col}_{key1}-{col}_{key2}"] = p_val
                        else:
                            p_values_smaller_threshold[f"{col}_{key1}-{col}_{key2}"] = p_val
    print("p_values_smaller_threshold:")
    print(p_values_smaller_threshold)
    print("p_values_greater_threshold:")
    print(p_values_greater_threshold)

## Bắt đầu phân tích

### Import dữ liệu

In [ ]:
raw_df = pd.read_excel("orange_data.xlsx", sheet_name=0)

In [ ]:
raw_df.head(10)

,Name,p,C NaOH,Khối lượng(g),Đường kính ngang(mm),Đường kính dọc(mm),Độ đường (Brix %),TA (%),Brix:TA,BrmTA
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748


In [ ]:
raw_df.isnull().sum()

Name                    0
p                       0
C NaOH                  0
Khối lượng(g)           0
Đường kính ngang(mm)    0
Đường kính dọc(mm)      0
Độ đường (Brix %)       0
TA (%)                  0
Brix:TA                 0
BrmTA                   0
dtype: int64

#### Nhận định
- Không có dữ liệu null.

### Chuẩn hóa tên cột

In [ ]:
raw_df.columns

Index(['Name', 'p', 'C NaOH', 'Khối lượng(g)', 'Đường kính ngang(mm)',
       'Đường kính dọc(mm)', 'Độ đường (Brix %)', 'TA (%)', 'Brix:TA',
       'BrmTA'],
      dtype='object')

In [ ]:
new_column_names = {
    "Name": "name",
    "p": "v_naoh",
    "C NaOH": "c_naoh",
    "Khối lượng(g)": "mass",
    "Đường kính ngang(mm)": "h_diameter",
    "Đường kính dọc(mm)": "v_diameter",
    "Độ đường (Brix %)": "brix",
    "TA (%)": "ta",
    "Brix:TA": "brix_ta",
    "BrmTA": "brm_ta",
}

std_col_name_dataframe = raw_df.rename(columns=new_column_names)

In [ ]:
# pd.set_option('display.max_rows', 20)
pd.set_option('display.max_rows', None)
std_col_name_dataframe

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
0,A1,1.90,0.4000,255,257,262,10.90,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.80,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.00,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.00,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.90,3.073984,3.545887,7.826016
5,A7,1.30,0.4128,357,305,290,10.70,3.436714,3.113439,7.263286
6,A8,2.00,0.4000,309,275,285,11.20,5.123307,2.186088,6.076693
7,A9,1.90,0.4128,321,285,280,9.20,5.022890,1.831615,4.177110
8,A10,1.45,0.4082,245,255,259,9.50,3.790542,2.506238,5.709458
9,A11,2.00,0.4128,275,267,270,10.50,5.287252,1.985909,5.212748


#### Nhận định
- Dữ liệu gồm 81 bản ghi và 10 trường dữ liệu
- Cột "name" có nhãn ghi tiền tố A, B, C
=> Có thể dữ liệu được chia thành 3 nhóm A, B, C

### Khảo sát dữ liệu

In [ ]:
std_col_name_dataframe.shape

(81, 10)

In [ ]:
std_col_name_dataframe.describe()

,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,1.477778,0.406857,222.197531,241.777778,241.358025,11.169259,3.849815,3.024935,7.319445
std,0.297069,0.007092,50.582215,23.652167,22.587225,1.362702,0.774895,0.786520,1.587514
min,0.800000,0.400000,126.000000,195.000000,197.000000,7.400000,2.057520,1.795715,4.076693
25%,1.300000,0.401600,179.000000,222.000000,223.000000,10.400000,3.370111,2.512846,6.148368
50%,1.400000,0.404800,228.000000,242.000000,242.000000,11.100000,3.701077,2.920633,7.332125
75%,1.600000,0.408200,255.000000,257.000000,255.000000,12.300000,4.229802,3.371688,8.199505
max,2.300000,0.423000,357.000000,305.000000,290.000000,14.000000,5.891803,6.609899,11.542480


#### Nhận định
- c_naoh có độ lệch chuẩn rất nhỏ, thể hiện rằng
dữ liệu này gần như không thay đổi trong cả bộ
dữ liệu

### Trực quan hóa dữ liệu

#### Lưu dữ liệu vào dictionary list_dataset
Tạo ra một Dataset để lưu dataframe "std_col_name_dataframe"

- Đặt tên là "orange_std_col_name" (orange stander colum names)
- dataframe: std_col_name_dataframe
- Kiểu scaler của dataframe này là ScalerType.Raw_ (chưa scale)

In [ ]:
list_dataset["orange_std_col_name"] = Dataset(
    "orange_std_col_name", std_col_name_dataframe, ScalerType.Raw_
)

In [ ]:
list_dataset["orange_std_col_name"].get_dataframe()

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
0,A1,1.90,0.4000,255,257,262,10.90,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.80,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.00,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.00,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.90,3.073984,3.545887,7.826016
5,A7,1.30,0.4128,357,305,290,10.70,3.436714,3.113439,7.263286
6,A8,2.00,0.4000,309,275,285,11.20,5.123307,2.186088,6.076693
7,A9,1.90,0.4128,321,285,280,9.20,5.022890,1.831615,4.177110
8,A10,1.45,0.4082,245,255,259,9.50,3.790542,2.506238,5.709458
9,A11,2.00,0.4128,275,267,270,10.50,5.287252,1.985909,5.212748


#### Plot dữ liệu cho các biến định lượng

In [ ]:
list_dataset["orange_std_col_name"].get_numeric_cols()

['v_naoh',
 'c_naoh',
 'mass',
 'h_diameter',
 'v_diameter',
 'brix',
 'ta',
 'brix_ta',
 'brm_ta']

Ta thấy có 9 cột nên ta có thể vẽ một ma trận
biểu đồ 3x3.

In [ ]:
# Đặt biến boolean để activate
if GENERATE_PLOTS:
    plot_numeric_along_dataframe(list_dataset["orange_std_col_name"], 3, 3, (12, 12))

#### Nhận định
- Đồ thị đường của dữ liệu "h_diameter", "v_diameter", "brix", "brm_ta"
chia thành 3 mức khá rõ ràng.
- Đồ thị đường của dữ liệu "mass" chỉ thấy chia thành 2 mức.
- Đồ thị đường của dữ liệu khác thì chưa thấy có pattern gì rõ ràng.

#### Plot histogram, KDE và boxplot của các biến định lượng

In [ ]:
if GENERATE_PLOTS:
    plot_hist_kde_box_all_columns(list_dataset["orange_std_col_name"])

In [ ]:
# kurtosis of normal == 0.0
list_dataset["orange_std_col_name"].get_dataframe().kurtosis(numeric_only=True)

v_naoh        0.108709
c_naoh        0.760385
mass         -0.312707
h_diameter   -0.416495
v_diameter   -0.641989
brix         -0.109207
ta            0.191656
brix_ta       4.801462
brm_ta       -0.047842
dtype: float64

#### Nhận định
- Các dữ liệu có Kutorsis nhỏ hơn 0 (normal distribution) cho thấy dữ liệu tập trung nhiều
quanh giá trị trung tâm.
- Do đó dữ liệu có xu hướng có ít ngoại lệ (outliers).
- Dữ liệu "brix_ta" có Kutorsis cho thấy dữ liệu phân bố nhiều về hai cực. Do đó khả năng cao
sẽ có nhiều giá trị ngoại lệ (outliers).

### Phân loại dữ liệu

Ta thấy nhãn A, B, C của dữ liệu có ý nghĩa phân nhóm.
Ta sẽ kiểm định xem dữ liệu này có thực sự chia làm 3 nhóm được không.

In [ ]:
print(list_dataset["orange_std_col_name"].get_dataframe()["name"].to_list())

['A1', 'A2', 'A3', 'A4', 'A5', 'A7', 'A8', 'A9', 'A10', 'A11', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B30', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C8', 'C9', 'C10', 'C11', 'C12', 'C14', 'C15', 'C16', 'C17', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C28', 'C29']


In [ ]:
import re

data_type = []
for name in list_dataset["orange_std_col_name"].get_dataframe()["name"].str.upper():
    data_type.append(re.findall("[A-Z]", name)[0])

data_type = pd.DataFrame(data_type, columns=["type"])
data_type["count"] = 1
data_type.groupby(["type"]).count()["count"]

type
A    26
B    30
C    25
Name: count, dtype: int64

=> Dữ liệu tương đối đồng đều giữa các nhóm.

In [ ]:
orange_std_col_name = list_dataset["orange_std_col_name"].get_dataframe()
pd.concat([orange_std_col_name, data_type["type"]], axis=1)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,A1,1.90,0.4000,255,257,262,10.90,4.867141,2.239508,6.032859,A
1,A2,1.20,0.4000,325,288,278,9.80,3.073984,3.188045,6.726016,A
2,A3,2.30,0.4000,185,235,237,11.00,5.891803,1.867001,5.108197,A
3,A4,1.40,0.4128,203,245,245,9.00,3.701077,2.431725,5.298923,A
4,A5,1.20,0.4000,229,253,249,10.90,3.073984,3.545887,7.826016,A
5,A7,1.30,0.4128,357,305,290,10.70,3.436714,3.113439,7.263286,A
6,A8,2.00,0.4000,309,275,285,11.20,5.123307,2.186088,6.076693,A
7,A9,1.90,0.4128,321,285,280,9.20,5.022890,1.831615,4.177110,A
8,A10,1.45,0.4082,245,255,259,9.50,3.790542,2.506238,5.709458,A
9,A11,2.00,0.4128,275,267,270,10.50,5.287252,1.985909,5.212748,A


In [ ]:
orange_std_col_name_with_type = pd.concat([orange_std_col_name, data_type["type"]], axis=1)

In [ ]:
list_dataset["orange_std_col_name_with_type"] = Dataset("orange_std_col_name_with_type", orange_std_col_name_with_type, ScalerType.Raw_)

In [ ]:
list_dataset["orange_std_col_name_with_type"].get_dataframe()

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,A1,1.90,0.4000,255,257,262,10.90,4.867141,2.239508,6.032859,A
1,A2,1.20,0.4000,325,288,278,9.80,3.073984,3.188045,6.726016,A
2,A3,2.30,0.4000,185,235,237,11.00,5.891803,1.867001,5.108197,A
3,A4,1.40,0.4128,203,245,245,9.00,3.701077,2.431725,5.298923,A
4,A5,1.20,0.4000,229,253,249,10.90,3.073984,3.545887,7.826016,A
5,A7,1.30,0.4128,357,305,290,10.70,3.436714,3.113439,7.263286,A
6,A8,2.00,0.4000,309,275,285,11.20,5.123307,2.186088,6.076693,A
7,A9,1.90,0.4128,321,285,280,9.20,5.022890,1.831615,4.177110,A
8,A10,1.45,0.4082,245,255,259,9.50,3.790542,2.506238,5.709458,A
9,A11,2.00,0.4128,275,267,270,10.50,5.287252,1.985909,5.212748,A


In [ ]:
list_dataset["orange_std_col_name_with_type"] = Dataset("orange_std_col_name_with_type", orange_std_col_name_with_type, ScalerType.Raw_)

In [ ]:
list_dataset["orange_std_col_name_with_type"].get_dataframe()

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,A1,1.90,0.4000,255,257,262,10.90,4.867141,2.239508,6.032859,A
1,A2,1.20,0.4000,325,288,278,9.80,3.073984,3.188045,6.726016,A
2,A3,2.30,0.4000,185,235,237,11.00,5.891803,1.867001,5.108197,A
3,A4,1.40,0.4128,203,245,245,9.00,3.701077,2.431725,5.298923,A
4,A5,1.20,0.4000,229,253,249,10.90,3.073984,3.545887,7.826016,A
5,A7,1.30,0.4128,357,305,290,10.70,3.436714,3.113439,7.263286,A
6,A8,2.00,0.4000,309,275,285,11.20,5.123307,2.186088,6.076693,A
7,A9,1.90,0.4128,321,285,280,9.20,5.022890,1.831615,4.177110,A
8,A10,1.45,0.4082,245,255,259,9.50,3.790542,2.506238,5.709458,A
9,A11,2.00,0.4128,275,267,270,10.50,5.287252,1.985909,5.212748,A


In [ ]:
if GENERATE_PLOTS:
    plot_all_columns_group_by_a_categorical_col(list_dataset["orange_std_col_name_with_type"], "type")

#### Nhận định
- Các dữ liệu của cột "mass", "h_diameter", "v_diameter" và "brm_ta" được phân loại khá rõ theo các nhóm A, B, C.

#### Thực hiện kiểm định để xem có sự khác nhau giữa 3 nhóm A, B, C hay không.
Kiểm định sử dụng: Welch's t-test (kiểm định t-test cho các nhóm dữ liệu có số quan sát khác nhau).

In [ ]:
partition_dataframes = divide_dataframe_with_categorical_col(list_dataset["orange_std_col_name_with_type"], "type")

In [ ]:
for group in partition_dataframes.keys():
    print(f"{group}: {partition_dataframes[group].shape}")

A: (26, 11)
B: (30, 11)
C: (25, 11)


In [ ]:
partition_dataframes["A"].head(30)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859,A
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016,A
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197,A
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923,A
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016,A
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286,A
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693,A
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110,A
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458,A
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748,A


In [ ]:
partition_dataframes["B"].head(30)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
26,B1,1.10,0.4048,232,242,240,9.00,2.851632,3.156087,6.148368,B
27,B2,1.25,0.4048,244,245,250,10.15,3.240491,3.132241,6.909509,B
28,B3,1.70,0.4048,256,252,248,11.00,4.407068,2.495990,6.592932,B
29,B4,1.55,0.4048,228,242,238,11.10,4.018209,2.762424,7.081791,B
30,B5,1.40,0.4048,249,248,247,11.00,3.629350,3.030845,7.370650,B
31,B6,1.35,0.4048,213,234,235,11.30,3.499731,3.228820,7.800269,B
32,B7,1.35,0.4048,254,253,251,11.20,3.499731,3.200246,7.700269,B
33,B8,1.60,0.4048,232,238,242,12.40,4.147829,2.989516,8.252171,B
34,B9,1.20,0.4048,214,235,236,10.50,3.110872,3.375260,7.389128,B
35,B10,1.55,0.4048,267,257,250,11.00,4.018209,2.737538,6.981791,B


In [ ]:
partition_dataframes["C"].head(30)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
56,C1,1.20,0.4230,175,215,223,13.00,3.250738,3.999092,9.749262,C
57,C2,1.30,0.4230,146,202,207,12.80,3.521633,3.634678,9.278367,C
58,C3,1.25,0.4230,182,220,225,11.40,3.386185,3.366620,8.013815,C
59,C4,1.00,0.4230,179,221,222,12.80,2.708948,4.725081,10.091052,C
60,C5,1.80,0.4016,173,224,215,12.30,4.629420,2.656920,7.670580,C
61,C6,2.15,0.4230,173,222,215,11.30,5.824239,1.940168,5.475761,C
62,C8,1.00,0.4230,133,199,203,12.30,2.708948,4.540507,9.591052,C
63,C9,1.90,0.4230,179,220,220,12.50,5.147002,2.428598,7.352998,C
64,C10,1.75,0.4230,173,216,220,12.90,4.740660,2.721140,8.159340,C
65,C11,1.30,0.4230,185,222,223,13.40,3.521633,3.805053,9.878367,C


So sánh 3 nhóm dữ liệu theo "mass", "h_diameter", "v_diameter" và "brix"

In [ ]:
columns_to_compare = ["mass", "h_diameter", "v_diameter", "brix"]

In [ ]:
calculate_p_values_between_partitions(partition_dataframes, columns_to_compare, 0.05)

p_value threshold:  0.05
p_values_smaller_threshold:
{'mass_A-mass_B': 0.0001489890255481789, 'h_diameter_A-h_diameter_B': 2.108803554571748e-08, 'v_diameter_A-v_diameter_B': 2.60539852542002e-10, 'brix_A-brix_B': 0.006463772327166934, 'mass_A-mass_C': 8.345760914262529e-16, 'h_diameter_A-h_diameter_C': 1.64191506004629e-19, 'v_diameter_A-v_diameter_C': 3.860228042999967e-20, 'brix_A-brix_C': 2.3428541465169138e-09, 'mass_B-mass_A': 0.0001489890255481789, 'h_diameter_B-h_diameter_A': 2.108803554571748e-08, 'v_diameter_B-v_diameter_A': 2.60539852542002e-10, 'brix_B-brix_A': 0.006463772327166934, 'mass_B-mass_C': 2.317855817307947e-16, 'h_diameter_B-h_diameter_C': 8.701558097113777e-17, 'v_diameter_B-v_diameter_C': 9.855593390467438e-17, 'brix_B-brix_C': 0.0002194130970339367, 'mass_C-mass_A': 8.345760914262529e-16, 'h_diameter_C-h_diameter_A': 1.64191506004629e-19, 'v_diameter_C-v_diameter_A': 3.860228042999967e-20, 'brix_C-brix_A': 2.3428541465169138e-09, 'mass_C-mass_B': 2.31785581730

#### Nhận định
- Sau khi thực hiện kiểm định, ta thấy p_value của các giá trị mean của 3 nhóm A, B, C đều nhỏ hơn 0.05.
- => Do đó, kết quả này có ý nghĩa thống kê.
- => Dữ liệu được chia nhóm thành A, B, C.

### Khảo sát tương quan dữ liệu

#### Pairplots

In [ ]:
if GENERATE_PLOTS:
    plt.figure(figsize=(16, 16))
    sns.pairplot(list_dataset["orange_std_col_name_with_type"].get_dataframe())

#### Nhận định
- Với biến phản hồi là "brix", ta thấy có sự tương quan tuyến tính với các biến giải thích định lượng: "mass", "h_diameter", "v_diameter".

#### Heatmap

In [ ]:
if GENERATE_PLOTS:
    plt.figure(figsize=(15, 15))
    dataset_corr = list_dataset["orange_std_col_name_with_type"].get_numeric_dataframe().corr()
    sns.heatmap(dataset_corr, annot=True)

#### Nhận định
- Với biến phản hồi là "brix", ta thấy có sự tương quan tuyến tính với các biến giải thích định lượng:
"mass", "h_diameter", "v_diameter" với hệ số correlation tương ứng là -0.45, -0.52, -0.52

### Phân tích hồi quy đơn biến

In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from typing import Optional, Callable


def summary_linear_model(x: list, y: list, dataset: Dataset):
    # Convert one-hot vector
    dataframe = dataset.get_dataframe()
    for col in x:
        if isinstance(dataframe[col][0], (bool, np.bool_)):
            dataframe[col] = dataframe[col].astype(int)

    x_ = sm.add_constant(dataframe[x])
    y_ = dataframe[y]
    results = sm.OLS(y_, x_).fit()
    Intercept = results.params.iloc[0]
    Slope = results.params.iloc[1:]
    P_values = results.pvalues[1:]
    R_squared = results.rsquared
    MSE = mean_squared_error(y_, results.predict())
    print("Intercept\n", Intercept, end="\n\n")
    print("Slope\n", Slope, end="\n\n")
    print("P_values\n", P_values, end="\n\n")
    print("R_squared\n", R_squared, end="\n\n")
    print("MSE\n", MSE, end="\n\n")
    return {
        "Intercept": Intercept,
        "Slope": Slope,
        "P_values": P_values,
        "R_squared": R_squared,
        "MSE": MSE,
    }

#### Phân tích biến "mass"

In [ ]:
summary_linear_model(["mass"], ["brix"], list_dataset["orange_std_col_name_with_type"])

Intercept
 13.858654422770524

Slope
 mass   -0.012104
dtype: float64

P_values
 mass    0.000026
dtype: float64

R_squared
 0.20184808679624422

MSE
 1.4638357905632924



{'Intercept': 13.858654422770524,
 'Slope': mass   -0.012104
 dtype: float64,
 'P_values': mass    0.000026
 dtype: float64,
 'R_squared': 0.20184808679624422,
 'MSE': 1.4638357905632924}

#### Nhận định
- p_value << 0.05 => Biến "mass" có ý nghĩa thống kê với mô hình hồi quy tuyến tính.

#### Phân tích biến "v_diameter"

In [ ]:
summary_linear_model(["v_diameter"], ["brix"], list_dataset["orange_std_col_name_with_type"])

Intercept
 18.71561067446182

Slope
 v_diameter   -0.031266
dtype: float64

P_values
 v_diameter    7.214880e-07
dtype: float64

R_squared
 0.2685804964678721

MSE
 1.3414464458134248



{'Intercept': 18.71561067446182,
 'Slope': v_diameter   -0.031266
 dtype: float64,
 'P_values': v_diameter    7.214880e-07
 dtype: float64,
 'R_squared': 0.2685804964678721,
 'MSE': 1.3414464458134248}

#### Nhận định
- p_value << 0.05 => Biến "v_diameter" có ý nghĩa thống kê với mô hình hồi quy tuyến tính.

#### Phân tích biến "h_diameter"

In [ ]:
summary_linear_model(["h_diameter"], ["brix"], list_dataset["orange_std_col_name_with_type"])

Intercept
 18.41872089231833

Slope
 h_diameter   -0.029984
dtype: float64

P_values
 h_diameter    6.358985e-07
dtype: float64

R_squared
 0.27084372901517995

MSE
 1.3372956059165202



{'Intercept': 18.41872089231833,
 'Slope': h_diameter   -0.029984
 dtype: float64,
 'P_values': h_diameter    6.358985e-07
 dtype: float64,
 'R_squared': 0.27084372901517995,
 'MSE': 1.3372956059165202}

#### Nhận định
- p_value << 0.05 => Biến "h_diameter" có ý nghĩa thống kê với mô hình hồi quy tuyến tính.

### Phân tích hồi quy

#### Phân tích biến "mass", "v_diameter"

In [ ]:
summary_linear_model(["mass", "v_diameter"], ["brix"], list_dataset["orange_std_col_name_with_type"])

Intercept
 25.981878092136817

Slope
 mass          0.024066
v_diameter   -0.083527
dtype: float64

P_values
 mass          0.022395
v_diameter    0.000537
dtype: float64

R_squared
 0.3161745734278585

MSE
 1.2541574070724175



{'Intercept': 25.981878092136817,
 'Slope': mass          0.024066
 v_diameter   -0.083527
 dtype: float64,
 'P_values': mass          0.022395
 v_diameter    0.000537
 dtype: float64,
 'R_squared': 0.3161745734278585,
 'MSE': 1.2541574070724175}

#### Nhận định
- Các giá trị p_value < 0.05 => Có thể đưa biến giải thích "mass" và "v_diameter" vào mô hình.

#### Phân tích biến "mass", "h_diameter"

In [ ]:
summary_linear_model(["mass", "h_diameter"], ["brix"], list_dataset["orange_std_col_name_with_type"])

Intercept
 29.447261559705822

Slope
 mass          0.038838
h_diameter   -0.111292
dtype: float64

P_values
 mass          0.001728
h_diameter    0.000041
dtype: float64

R_squared
 0.3575939425902803

MSE
 1.178192977244598



{'Intercept': 29.447261559705822,
 'Slope': mass          0.038838
 h_diameter   -0.111292
 dtype: float64,
 'P_values': mass          0.001728
 h_diameter    0.000041
 dtype: float64,
 'R_squared': 0.3575939425902803,
 'MSE': 1.178192977244598}

#### Nhận định
- Các giá trị p_value < 0.05 => Có thể đưa biến giải thích "mass" và "h_diameter" vào mô hình.

#### Phân tích biến "v_diameter", "h_diameter"

In [ ]:
summary_linear_model(["v_diameter", "h_diameter"], ["brix"], list_dataset["orange_std_col_name_with_type"])

Intercept
 18.640212018418648

Slope
 v_diameter   -0.013381
h_diameter   -0.017542
dtype: float64

P_values
 v_diameter    0.601662
h_diameter    0.473980
dtype: float64

R_squared
 0.2734029409332125

MSE
 1.3326019305155372



{'Intercept': 18.640212018418648,
 'Slope': v_diameter   -0.013381
 h_diameter   -0.017542
 dtype: float64,
 'P_values': v_diameter    0.601662
 h_diameter    0.473980
 dtype: float64,
 'R_squared': 0.2734029409332125,
 'MSE': 1.3326019305155372}

#### Nhận định
- Khi đưa 2 biến "h_diamter" và "v_diameter" vào mô hình, ta thấy p_value rất cao so với 0.05.
- Nhưng khi 2 biến này đứng riêng thì lại cho p_value thấp.
- => Ta thấy 2 biến "h_diamter" và "v_diameter" có tính cộng tuyến (collinearity)
nên ta sẽ chỉ lấy một trong hai biến này.
- Vì chỉ số r2 của mô hình có biến "mass" và "v_diameter" cao hơn (r2 = 0.3575), nên ta sẽ
chọn 2 biến này để đưa vào mô hình và loại bỏ biến "h_diameter" khỏi mô hình.

#### Phân tích biến "mass", "v_diamter", "type"

Trước tiên, ta phải chuyển biến định danh "type" thành dạng one-hot vector.

In [ ]:
list_dataset["orange_std_col_name_ohv_type"] = list_dataset["orange_std_col_name_with_type"].get_one_hot_vectorized_dataset("type")

orange_std_col_name_with_type_ohv_type


In [ ]:
summary_linear_model(["mass", "v_diameter", "A"], ["brix"], list_dataset["orange_std_col_name_ohv_type"])

Intercept
 24.661947470498433

Slope
 mass          0.021512
v_diameter   -0.075491
A            -0.162455
dtype: float64

P_values
 mass          0.105456
v_diameter    0.030985
A             0.751400
dtype: float64

R_squared
 0.317071055493232

MSE
 1.252513230680452



{'Intercept': 24.661947470498433,
 'Slope': mass          0.021512
 v_diameter   -0.075491
 A            -0.162455
 dtype: float64,
 'P_values': mass          0.105456
 v_diameter    0.030985
 A             0.751400
 dtype: float64,
 'R_squared': 0.317071055493232,
 'MSE': 1.252513230680452}

#### Nhận định
- Khi đưa thêm biến "A" vào mô hình có sẵn biến "mass" và "v_diameter",
ta thấy p_value của cột "A" rất cao so với 0.05.
- => Ta không cần biến "A" cho mô hình.

In [ ]:
summary_linear_model(["mass", "v_diameter", "B"], ["brix"], list_dataset["orange_std_col_name_ohv_type"])

Intercept
 28.973698656383043

Slope
 mass          0.033633
v_diameter   -0.103954
B            -0.506306
dtype: float64

P_values
 mass          0.004995
v_diameter    0.000129
B             0.091168
dtype: float64

R_squared
 0.3412119413996253

MSE
 1.2082380842815204



{'Intercept': 28.973698656383043,
 'Slope': mass          0.033633
 v_diameter   -0.103954
 B            -0.506306
 dtype: float64,
 'P_values': mass          0.004995
 v_diameter    0.000129
 B             0.091168
 dtype: float64,
 'R_squared': 0.3412119413996253,
 'MSE': 1.2082380842815204}

#### Nhận định
- Khi đưa thêm biến "B" vào mô hình có sẵn biến "mass" và "v_diameter",
ta thấy p_value của cột "B" khá cao so với 0.05.
- => Ta không cần biến "B" cho mô hình.

In [ ]:
summary_linear_model(["mass", "v_diameter", "C"], ["brix"], list_dataset["orange_std_col_name_ohv_type"])

Intercept
 22.052217850506945

Slope
 mass          0.027951
v_diameter   -0.072385
C             1.221981
dtype: float64

P_values
 mass          0.006356
v_diameter    0.001845
C             0.004915
dtype: float64

R_squared
 0.3833461525940429

MSE
 1.1309626116137055



{'Intercept': 22.052217850506945,
 'Slope': mass          0.027951
 v_diameter   -0.072385
 C             1.221981
 dtype: float64,
 'P_values': mass          0.006356
 v_diameter    0.001845
 C             0.004915
 dtype: float64,
 'R_squared': 0.3833461525940429,
 'MSE': 1.1309626116137055}

#### Nhận định
- Khi đưa thêm biến "C" vào mô hình có sẵn biến "mass" và "v_diameter",
ta thấy p_value của cột "C" rất nhỏ so với 0.05.
- => Đưa biến "C" vào mô hình.

#### Tổng kết Phân tích hồi quy
- Biến giải thích: "mass", "v_diameter", "C"
- Biến phản hồi: "brix"

In [ ]:
list_dataset["orange_std_col_name_ohv_type"].get_dataframe()

,A,B,C,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,1,0,0,A1,1.90,0.4000,255,257,262,10.90,4.867141,2.239508,6.032859,A
1,1,0,0,A2,1.20,0.4000,325,288,278,9.80,3.073984,3.188045,6.726016,A
2,1,0,0,A3,2.30,0.4000,185,235,237,11.00,5.891803,1.867001,5.108197,A
3,1,0,0,A4,1.40,0.4128,203,245,245,9.00,3.701077,2.431725,5.298923,A
4,1,0,0,A5,1.20,0.4000,229,253,249,10.90,3.073984,3.545887,7.826016,A
5,1,0,0,A7,1.30,0.4128,357,305,290,10.70,3.436714,3.113439,7.263286,A
6,1,0,0,A8,2.00,0.4000,309,275,285,11.20,5.123307,2.186088,6.076693,A
7,1,0,0,A9,1.90,0.4128,321,285,280,9.20,5.022890,1.831615,4.177110,A
8,1,0,0,A10,1.45,0.4082,245,255,259,9.50,3.790542,2.506238,5.709458,A
9,1,0,0,A11,2.00,0.4128,275,267,270,10.50,5.287252,1.985909,5.212748,A


## Xây dựng mô hình

### Mô hình cơ bản (LINEAR_REGRESSION, ScalerType.Raw_)

In [ ]:
# Trước tiên, chúng ta khai báo
# Cột x gồm những cột nào (dưới dạng list)
# Cột y gồm những cột nào (dưới dạng list)
# Khi đưa dataset vào, kể cả dataset có những cột khác cũng không sao
# vì code sẽ tự động trích xuất các cột cần thiết và loại bỏ
# những cột khác.

result_frame.reset_result()

x_cols = ["mass", "v_diameter", "C"]
y_cols = ["brix"]

model = Model(
    dataset_name="orange_std_col_name_mass_v_C",
    pretrained_model_features=[],
    model_type=ModelType.LINEAR_REGRESSION,
    x_cols=x_cols,
    y_cols=y_cols,
    dataset=list_dataset["orange_std_col_name_ohv_type"],
    scaler_type=ScalerType.Raw_,
)

result_frame.add_result(model.evaluate())

result_frame.display_result()

100%|████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 1164.52it/s]

Training | ModelType: ModelType.LINEAR_REGRESSION | Datasetname: orange_std_col_name_mass_v_C | ScalerType: ScalerType.Raw_ | Params: {}
R2: 0.3141374307465781 | MSE: 1.257893591021877


,dataset_name,pretrained_model_features,model_type,scaler_type,r2,mse,params
0,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,Raw_,0.314137,1.257894,{}


In [ ]:
result_frame.save_result("result_1.xlsx")

#### Nhận định
- Mô hình LINEAR_REGRESSION với scaler của data là Raw_ cho kết quả
- r2 = 0.314137
- mse = 1.257894
- => Đây là kết quả với mô hình cơ bản nhất.
- => Kết quả này chưa tốt. Ta cần có những phương pháp khác để cải thiện dữ liệu / mô hình.

### Mô hình với dữ liệu đã chuẩn hóa (LINEAR_REGRESSION, tất cả ScalerType)

In [ ]:
# Trước tiên, chúng ta khai báo
# Cột x gồm những cột nào (dưới dạng list)
# Cột y gồm những cột nào (dưới dạng list)
# Khi đưa dataset vào, kể cả dataset có những cột khác cũng không sao
# vì code sẽ tự động trích xuất các cột cần thiết và loại bỏ
# những cột khác.

result_frame.reset_result()

x_cols = ["mass", "v_diameter", "C"]
y_cols = ["brix"]

for scaler_type in ScalerType:
    model = Model(
        dataset_name="orange_std_col_name_mass_v_C",
        pretrained_model_features=[],
        model_type=ModelType.LINEAR_REGRESSION,
        x_cols=x_cols,
        y_cols=y_cols,
        dataset=list_dataset["orange_std_col_name_ohv_type"],
        scaler_type=scaler_type,
    )

    result_frame.add_result(model.evaluate())

result_frame.display_result()

100%|████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 1276.01it/s]


Training | ModelType: ModelType.LINEAR_REGRESSION | Datasetname: orange_std_col_name_mass_v_C | ScalerType: ScalerType.Raw_ | Params: {}
R2: 0.3141374307465781 | MSE: 1.257893591021877


100%|█████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 844.49it/s]


Training | ModelType: ModelType.LINEAR_REGRESSION | Datasetname: orange_std_col_name_mass_v_C | ScalerType: ScalerType.MaxAbsScaler_ | Params: {}
R2: 0.31413743074657796 | MSE: 1.2578935910218771


100%|█████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 859.77it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 527.53it/s]


Training | ModelType: ModelType.LINEAR_REGRESSION | Datasetname: orange_std_col_name_mass_v_C | ScalerType: ScalerType.RobustScaler_ | Params: {}
R2: 0.3141374307465782 | MSE: 1.257893591021877


100%|█████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 774.14it/s]

Training | ModelType: ModelType.LINEAR_REGRESSION | Datasetname: orange_std_col_name_mass_v_C | ScalerType: ScalerType.StandardScaler_ | Params: {}
R2: 0.31413743074657785 | MSE: 1.2578935910218776


,dataset_name,pretrained_model_features,model_type,scaler_type,r2,mse,params
2,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,MinMaxScaler_,0.314137,1.257894,{}
4,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,RobustScaler_,0.314137,1.257894,{}
0,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,Raw_,0.314137,1.257894,{}
1,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,MaxAbsScaler_,0.314137,1.257894,{}
5,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,StandardScaler_,0.314137,1.257894,{}
3,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,Normalizer_,0.231867,1.408780,{}


In [ ]:
result_frame.save_result("result_2.xlsx")

#### Nhận định
- Với cùng mô hình LINEAR_REGRESSION, các loại scaler vẫn cho kết quả tương tự
(scaler Normalizer_ cho kết quả kém hơn).
- Như vậy, dữ liệu có thể để ở dạng Raw_ (chưa chuẩn hóa).
- Nhiệm vụ bây giờ là ta cần tìm cách cải tiến mô hình.

### Các mô hình với dữ liệu đã chuẩn hóa (tất cả Model, tất cả ScalerType)

In [ ]:
# Trước tiên, chúng ta khai báo
# Cột x gồm những cột nào (dưới dạng list)
# Cột y gồm những cột nào (dưới dạng list)
# Khi đưa dataset vào, kể cả dataset có những cột khác cũng không sao
# vì code sẽ tự động trích xuất các cột cần thiết và loại bỏ
# những cột khác.

result_frame.reset_result()

x_cols = ["mass", "v_diameter", "C"]
y_cols = ["brix"]

for model_type in ModelType:
    for scaler_type in ScalerType:
        model = Model(
            dataset_name="orange_std_col_name_mass_v_C",
            pretrained_model_features=[],
            model_type=model_type,
            x_cols=x_cols,
            y_cols=y_cols,
            dataset=list_dataset["orange_std_col_name_ohv_type"],
            scaler_type=scaler_type,
        )

        result_frame.add_result(model.evaluate())

result_frame.display_result()

100%|████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 1244.31it/s]


Training | ModelType: ModelType.LINEAR_REGRESSION | Datasetname: orange_std_col_name_mass_v_C | ScalerType: ScalerType.Raw_ | Params: {}
R2: 0.3141374307465781 | MSE: 1.257893591021877


100%|█████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 881.74it/s]


Training | ModelType: ModelType.LINEAR_REGRESSION | Datasetname: orange_std_col_name_mass_v_C | ScalerType: ScalerType.MaxAbsScaler_ | Params: {}
R2: 0.31413743074657796 | MSE: 1.2578935910218771


100%|█████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 836.74it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 59.68it/s]


Training | ModelType: ModelType.XGB | Datasetname: orange_std_col_name_mass_v_C | ScalerType: ScalerType.RobustScaler_ | Params: {'objective': 'reg:linear', 'n_estimators': 10, 'seed': 123, 'verbosity': 0}
R2: -0.03806611934102877 | MSE: 1.90384601392871


100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 66.49it/s]

Training | ModelType: ModelType.XGB | Datasetname: orange_std_col_name_mass_v_C | ScalerType: ScalerType.StandardScaler_ | Params: {'objective': 'reg:linear', 'n_estimators': 10, 'seed': 123, 'verbosity': 0}
R2: -0.03806611934102877 | MSE: 1.90384601392871


,dataset_name,pretrained_model_features,model_type,scaler_type,r2,mse,params
2,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,MinMaxScaler_,0.314137,1.257894,{}
4,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,RobustScaler_,0.314137,1.257894,{}
0,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,Raw_,0.314137,1.257894,{}
1,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,MaxAbsScaler_,0.314137,1.257894,{}
5,orange_std_col_name_mass_v_C,[],LINEAR_REGRESSION,StandardScaler_,0.314137,1.257894,{}
8,orange_std_col_name_mass_v_C,[],SVR,MinMaxScaler_,0.300024,1.283779,"{'kernel': 'rbf', 'degree': 3, 'gamma': 'scale..."
11,orange_std_col_name_mass_v_C,[],SVR,StandardScaler_,0.283407,1.314255,"{'kernel': 'rbf', 'degree': 3, 'gamma': 'scale..."
7,orange_std_col_name_mass_v_C,[],SVR,MaxAbsScaler_,0.282051,1.316742,"{'kernel': 'rbf', 'degree': 3, 'gamma': 'scale..."
10,orange_std_col_name_mass_v_C,[],SVR,RobustScaler_,0.280330,1.319897,"{'kernel': 'rbf', 'degree': 3, 'gamma': 'scale..."
28,orange_std_col_name_mass_v_C,[],LIGHT_GBM,RobustScaler_,0.267041,1.344271,"{'boosting_type': 'gbdt', 'num_leaves': 31, 'm..."


In [ ]:
result_frame.save_result("result_3.xlsx")

#### Nhận định
- Sau khi thử tổ hợp các mô hình và các scaler, ta thấy kết quả tốt nhất vẫn là kết quả ban đầu.
- Mô hình: LINEAR_REGRESSION | Scaler: MinMaxScaler_ | r2: 0.314137
- Mô hình: LINEAR_REGRESSION | Scaler: RobustScaler_ | r2: 0.314137
- Mô hình: LINEAR_REGRESSION | Scaler: Raw_ | r2: 0.314137
- => Cần có phương án khác để cải thiện kết quả !